Hosting of my chess program on lichess, making use of the free Google Colab GPUs.

In [ ]:
!pip install chess

import numpy as np
import torch
import torch.nn as nn
import chess
import re

from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

# configuring device
try:
    device = xm.xla_device()
    print("Running on the TPU")
except:
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print('Running on the GPU')
        torch.cuda.synchronize()
    else:
        device = torch.device('cpu')
        print('Running on the CPU')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=274d9294a08ba9360b91818954d945275a3a02c044546cfa1ffdb7d1a534bc0b
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
Mounted at /content/drive
Running on the GPU


In [ ]:
# Variable initialisation if needed.

In [ ]:
# Setup lichess-bot.
!git clone https://github.com/walterliu417/lichess-bot-parrot.git
!pip install -r "lichess-bot-parrot/requirements.txt"

import os
os.chdir("lichess-bot-parrot")

Cloning into 'lichess-bot-parrot'...
remote: Enumerating objects: 5084, done.
remote: Counting objects: 100% (534/534), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 5084 (delta 384), reused 268 (delta 260), pack-reused 4550 (from 5)
Receiving objects: 100% (5084/5084), 1.66 MiB | 1.44 MiB/s, done.
Resolving deltas: 100% (3377/3377), done.


In [ ]:
lichessapi = userdata.get("lichessapi")

config = f'''token: "{lichessapi}"    # Lichess OAuth2 Token.
url: "https://lichess.org/"        # Lichess base URL.

engine:                            # Engine settings.
  dir: "./engines/"                # Directory containing the engine. This can be an absolute path or one relative to lichess-bot/.
  name: "Parrot"              # Binary name of the engine to use.
  interpreter: "python"
# interpreter_options:
#   - "-jar"
  working_dir: ""                  # Directory where the chess engine will read and write files. If blank or missing, the current directory is used.
                                   # NOTE: If working_dir is set, the engine will look for files and directories relative to this directory, not where lichess-bot was launched. Absolute paths are unaffected.
  protocol: "homemade"                  # "uci", "xboard" or "homemade"
  ponder: false                     # Think on opponent's time.

  polyglot:
    enabled: false                 # Activate polyglot book.
    book:
      standard:                    # List of book file paths for variant standard.
        - engines/book1.bin
        - engines/book2.bin
#     atomic:                      # List of book file paths for variant atomic.
#       - engines/atomicbook1.bin
#       - engines/atomicbook2.bin
#     etc.
#     Use the same pattern for 'chess960', 'giveaway' (antichess), 'crazyhouse', 'horde', 'kingofthehill', 'racingkings' and '3check' as well.
    min_weight: 1                  # Does not select moves with weight below min_weight (min 0, max: 65535).
    selection: "weighted_random"   # Move selection is one of "weighted_random", "uniform_random" or "best_move" (but not below the min_weight in the 2nd and 3rd case).
    max_depth: 20                  # How many moves from the start to take from the book.

  draw_or_resign:
    resign_enabled: false          # Whether or not the bot should resign.
    resign_score: -1000            # If the score is less than or equal to this value, the bot resigns (in cp).
    resign_for_egtb_minus_two: true # If true the bot will resign in positions where the online_egtb returns a wdl of -2.
    resign_moves: 3                # How many moves in a row the score has to be below the resign value.
    offer_draw_enabled: true       # Whether or not the bot should offer/accept draw.
    offer_draw_score: 0            # If the absolute value of the score is less than or equal to this value, the bot offers/accepts draw (in cp).
    offer_draw_for_egtb_zero: true # If true the bot will offer/accept draw in positions where the online_egtb returns a wdl of 0.
    offer_draw_moves: 10           # How many moves in a row the absolute value of the score has to be below the draw value.
    offer_draw_pieces: 10          # Only if the pieces on board are less than or equal to this value, the bot offers/accepts draw.

  online_moves:
    max_out_of_book_moves: 10      # Stop using online opening books after they don't have a move for 'max_out_of_book_moves' positions. Doesn't apply to the online endgame tablebases.
    max_retries: 2                 # The maximum amount of retries when getting an online move.
    # max_depth: 10                # How many moves from the start to take from online books. Default is no limit.
    chessdb_book:
      enabled: false               # Whether or not to use chessdb book.
      min_time: 20                 # Minimum time (in seconds) to use chessdb book.
      move_quality: "good"         # One of "all", "good", "best".
      min_depth: 20                # Only for move_quality: "best".
    lichess_cloud_analysis:
      enabled: false               # Whether or not to use lichess cloud analysis.
      min_time: 20                 # Minimum time (in seconds) the bot must have to use cloud analysis.
      move_quality: "best"         # One of "good", "best".
      max_score_difference: 50     # Only for move_quality: "good". The maximum score difference (in cp) between the best move and the other moves.
      min_depth: 20
      min_knodes: 0
    lichess_opening_explorer:
      enabled: false
      min_time: 20
      source: "masters"            # One of "lichess", "masters", "player"
      player_name: ""              # The lichess username. Leave empty for the bot's username to be used. Used only when source is "player".
      sort: "winrate"              # One of "winrate", "games_played"
      min_games: 10                # Minimum number of times a move must have been played to be chosen.
    online_egtb:
      enabled: false               # Whether or not to enable online endgame tablebases.
      min_time: 20                 # Minimum time (in seconds) the bot must have to use online EGTBs.
      max_pieces: 7                # Maximum number of pieces on the board to use endgame tablebases.
      source: "lichess"            # One of "lichess", "chessdb".
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).

  lichess_bot_tbs:                 # The tablebases list here will be read by lichess-bot, not the engine.
    syzygy:
      enabled: true               # Whether or not to use local syzygy endgame tablebases.
      paths:                       # Paths to Syzygy endgame tablebases.
        - "/content/drive/MyDrive/parrot/tablebase_5pc"
      max_pieces: 5                # Maximum number of pieces in the endgame tablebase.
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).
    gaviota:
      enabled: false               # Whether or not to use local gaviota endgame tablebases.
      paths:
        - "engines/gaviota"
      max_pieces: 5
      min_dtm_to_consider_as_wdl_1: 120 # The minimum DTM to consider as syzygy WDL=1/-1. Set to 100 to disable.
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).

# engine_options:                  # Any custom command line params to pass to the engine.
#   cpuct: 3.1

  homemade_options:
#   Hash: 256

  uci_options:                     # Arbitrary UCI options passed to the engine.
    Move Overhead: 100             # Increase if your bot flags games too often.
    Threads: 4                     # Max CPU threads the engine can use.
    Hash: 512                      # Max memory (in megabytes) the engine can allocate.
    SyzygyPath: "./syzygy/"        # Paths to Syzygy endgame tablebases that the engine reads.
    UCI_ShowWDL: true              # Show the chance of the engine winning.
#   go_commands:                   # Additional options to pass to the UCI go command.
#     nodes: 1                     # Search so many nodes only.
#     depth: 5                     # Search depth ply only.
#     movetime: 1000               # Integer. Search exactly movetime milliseconds.

# xboard_options:                  # Arbitrary XBoard options passed to the engine.
#   cores: "4"
#   memory: "4096"
#   egtpath:                       # Directory containing egtb (endgame tablabases), relative to this project. For 'xboard' engines.
#     gaviota: "Gaviota path"
#     nalimov: "Nalimov Path"
#     scorpio: "Scorpio Path"
#     syzygy: "Syzygy Path"
#   go_commands:                   # Additional options to pass to the XBoard go command.
#     depth: 5                     # Search depth ply only.
#     Do note that the go commands 'movetime' and 'nodes' are invalid and may cause bad time management for XBoard engines.

  silence_stderr: false            # Some engines (yes you, Leela) are very noisy.

abort_time: 30                     # Time to abort a game in seconds when there is no activity.
fake_think_time: false             # Artificially slow down the bot to pretend like it's thinking.
rate_limiting_delay: 0             # Time (in ms) to delay after sending a move to prevent "Too Many Requests" errors.
move_overhead: 2000                # Increase if your bot flags games too often.
max_takebacks_accepted: 0          # The number of times to allow an opponent to take back a move in a game.
quit_after_all_games_finish: false # If set to true, then pressing Ctrl-C to quit will only stop lichess-bot after all current games have finished.

correspondence:
  move_time: 60                    # Time in seconds to search in correspondence games.
  checkin_period: 300              # How often to check for opponent moves in correspondence games after disconnecting.
  disconnect_time: 150             # Time before disconnecting from a correspondence game.
  ponder: false                    # Ponder in correspondence games the bot is connected to.

challenge:                         # Incoming challenges.
  concurrency: 1                   # Number of games to play simultaneously.
  sort_by: "best"                  # Possible values: "best" and "first".
  preference: "none"               # Possible values: "none", "human", "bot".
  accept_bot: true                 # Accepts challenges coming from other bots.
  only_bot: false                  # Accept challenges by bots only.
  max_increment: 20                # Maximum amount of increment to accept a challenge in seconds. The max is 180. Set to 0 for no increment.
  min_increment: 0                 # Minimum amount of increment to accept a challenge in seconds.
  max_base: 10800                   # Maximum amount of base time to accept a challenge in seconds. The max is 10800 (3 hours).
  min_base: 180                      # Minimum amount of base time to accept a challenge in seconds.
  max_days: 14                     # Maximum number of days per move to accept a challenge for a correspondence game.
                                   # Unlimited games can be accepted by removing this field or specifying .inf
  min_days: 1                      # Minimum number of days per move to accept a challenge for a correspondence game.
  variants:                        # Chess variants to accept (https://lichess.org/variant).
    - standard
    - fromPosition
#   - antichess
#   - atomic
#   - chess960
#   - crazyhouse
#   - horde
#   - kingOfTheHill
#   - racingKings
#   - threeCheck
  time_controls:                   # Time controls to accept (bots are not allowed to play ultraBullet).
    - blitz
    - rapid
    - classical
#   - correspondence
  modes:                           # Game modes to accept.
    - casual                       # Unrated games.
    - rated                        # Rated games - must comment if the engine doesn't try to win.
# block_list:                      # List of users from which the challenges are always declined.
#   - user1
#   - user2
# allow_list:                      # List of users from which challenges are exclusively accepted, all others being declined. If empty, challenges from all users may be accepted.
#   - user3
#   - user4
# recent_bot_challenge_age: 60     # Maximum age of a bot challenge to be considered recent in seconds
# max_recent_bot_challenges: 2     # Maximum number of recent challenges that can be accepted from the same bot
  bullet_requires_increment: false # Require that bullet game challenges from bots have a non-zero increment
  max_simultaneous_games_per_user: 5  # Maximum number of simultaneous games with the same user

greeting:
  hello: "Hi! I'm {{me}}, a neural network based chess engine developed by YoMan417. Good luck!" # Message to send to opponent chat at the start of a game
  goodbye: "Good game!" # Message to send to opponent chat at the end of a game
  hello_spectators: "Hi! I'm {{me}}." # Message to send to spectator chat at the start of a game
  goodbye_spectators: "Thanks for watching!" # Message to send to spectator chat at the end of a game

# pgn_directory: "game_records"    # A directory where PGN-format records of the bot's games are kept
# pgn_file_grouping: "game"        # How to group games into files. Options are "game", "opponent", and "all"
                                   # "game" (default) - every game is written to a different file named "{{White name}} vs. {{Black name}} - {{lichess game ID}}.pgn"
                                   # "opponent" - every game with a given opponent is written to a file named "{{Bot name}} games vs. {{Opponent name}}.pgn"
                                   # "all" - every game is written to a single file named "{{Bot name}} games.pgn"

matchmaking:
  allow_matchmaking: true         # Set it to 'true' to challenge other bots.
  allow_during_games: false        # Set it to 'true' to create challenges during long games.
  challenge_variant: "random"      # If set to 'random', the bot will choose one variant from the variants enabled in 'challenge.variants'.
  challenge_timeout: 1            # Create a challenge after being idle for 'challenge_timeout' minutes. The minimum is 1 minute.
  challenge_initial_time:          # Initial time in seconds of the challenge (to be chosen at random).
    - 600
  challenge_increment:             # Increment in seconds of the challenge (to be chosen at random).
    - 1
    - 2
    - 3
    - 4
    - 5
#  challenge_days:                 # Days for correspondence challenge (to be chosen at random).
#    - 1
#    - 2
# opponent_min_rating: 600         # Opponents rating should be above this value (600 is the minimum rating in lichess).
# opponent_max_rating: 4000        # Opponents rating should be below this value (4000 is the maximum rating in lichess).
  opponent_rating_difference: 3000  # The maximum difference in rating between the bot's rating and opponent's rating.
  rating_preference: "none"        # One of "none", "high", "low".
  opponent_allow_tos_violation: false # Set to 'true' to allow challenging bots that violated the Lichess Terms of Service.
  challenge_mode: "random"         # Set it to the mode in which challenges are sent. Possible options are 'casual', 'rated' and 'random'.
  challenge_filter: none           # If a bot declines a challenge, do not issue a similar challenge to that bot. Possible options are 'none', 'coarse', and 'fine'.
# block_list:                      # The list of bots that will not be challenged
#   - user1
#   - user2
  include_challenge_block_list: false  # Do not challenge bots in the challenge: block_list in addition to the matchmaking block list.

# overrides:                       # List of overrides for the matchmaking specifications above. When a challenge is created, either the default specification above or one of the overrides will be randomly chosen.
#   bullet_only_horde:             # Name of the override. Can be anything as long as each override has a unique name ("bullet_only_horde" and "easy_chess960" in these examples).
#     challenge_variant: "horde"   # List of options to override. Only the options mentioned will change when making the challenge. The rest will follow the default matchmaking options above.
#     challenge_initial_time:
#       - 1
#       - 2
#     challenge_increment:
#       - 0
#       - 1
#
#   easy_chess960:
#     challenge_variant: "chess960"
#     opponent_min_rating: 400
#     opponent_max_rating: 1200
#     opponent_rating_difference:
#     challenge_mode: casual
#
#   no_pressure_correspondence:
#     challenge_initial_time:
#     challenge_increment:
#     challenge_days:
#       - 2
#       - 3
#     challenge_mode: casual
#
# The following configurations cannot be overridden: allow_matchmaking, challenge_timeout, challenge_filter and block_list.'''

with open("config.yml", "w") as file:
    file.write(config)

In [ ]:
!python lichess-bot.py


    .   _/|
    .  // o\
    .  || ._)  lichess-bot 2025.2.3.2 on Linux 6.1.85+
    .  //__\
    .  )___(   Play on Lichess with a bot
    
Checking engine configuration ...
Running on the GPU
5 piece Syzygy endgame tablebase found.
ComplexModel(
  (conv_net): Sequential(
    (Conv 1): Conv2d(1, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (SE 1): SE_Block(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (0): Linear(in_features=200, out_features=12, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=12, out_features=200, bias=False)
        (3): Sigmoid()
      )
    )
    (Batchnorm 1): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv activation): Mish()
    (Conv 2): Conv2d(200, 190, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (SE 2): SE_Block(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (0): Linear(in_fe

KeyboardInterrupt: 

In [ ]:
# These are debug blocks, to walk through the search and look for possible bugs.

from helperfuncs import *
from nn_creator import *
from tree import *


class SE_Block(nn.Module):
    "credits: https://github.com/moskomule/senet.pytorch/blob/master/senet/se_module.py#L4"
    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1, 1)
        return x * y.expand_as(x)

class ComplexModel(nn.Module):
  # Attempt at using a deeper CNN.

    def __init__(self, name):
        super().__init__()

        self.name = name

        self.conv_net = nn.Sequential()
        self.conv_net.add_module("Conv 1", nn.Conv2d(1, 200, 3, 1, 1))
        self.conv_net.add_module("SE 1", SE_Block(200))
        self.conv_net.add_module("Batchnorm 1", nn.BatchNorm2d(200))
        self.conv_net.add_module("Conv activation", nn.Mish())
        self.conv_net.add_module("Conv 2", nn.Conv2d(200, 190, 3, 1, 1))
        self.conv_net.add_module("SE 2", SE_Block(190))
        self.conv_net.add_module("Batchnorm 2", nn.BatchNorm2d(190))
        self.conv_net.add_module("Conv activation 2", nn.Mish())
        self.conv_net.add_module("Conv 3", nn.Conv2d(190, 180, 3, 1, 1))
        self.conv_net.add_module("SE 3", SE_Block(180))
        self.conv_net.add_module("Batchnorm 3", nn.BatchNorm2d(180))
        self.conv_net.add_module("Conv activation 3", nn.Mish())
        self.conv_net.add_module("Conv 4", nn.Conv2d(180, 170, 3, 1, 1))
        self.conv_net.add_module("SE 4", SE_Block(170))
        self.conv_net.add_module("Batchnorm 4", nn.BatchNorm2d(170))
        self.conv_net.add_module("Conv activation 4", nn.Mish())
        self.conv_net.add_module("Conv 5", nn.Conv2d(170, 160, 3, 1, 1))
        self.conv_net.add_module("SE 5", SE_Block(160))
        self.conv_net.add_module("Batchnorm 5", nn.BatchNorm2d(160))
        self.conv_net.add_module("Conv activation 5", nn.Mish())
        self.conv_net.add_module("Conv 6", nn.Conv2d(160, 150, 3, 1, 1))
        self.conv_net.add_module("SE 6", SE_Block(150))
        self.conv_net.add_module("Batchnorm 6", nn.BatchNorm2d(150))
        self.conv_net.add_module("Conv activation 6", nn.Mish())
        self.conv_net.add_module("Conv 7", nn.Conv2d(150, 140, 3, 1, 1))
        self.conv_net.add_module("SE 7", SE_Block(140))
        self.conv_net.add_module("Batchnorm 7", nn.BatchNorm2d(140))
        self.conv_net.add_module("Conv activation 7", nn.Mish())
        self.conv_net.add_module("Conv 8", nn.Conv2d(140, 130, 3, 1, 1))
        self.conv_net.add_module("SE 8", SE_Block(130))
        self.conv_net.add_module("Batchnorm 8", nn.BatchNorm2d(130))
        self.conv_net.add_module("Conv activation 8", nn.Mish())
        self.conv_net.add_module("Conv 9", nn.Conv2d(130, 120, 3, 1, 1))
        self.conv_net.add_module("SE 9", SE_Block(120))
        self.conv_net.add_module("Batchnorm 9", nn.BatchNorm2d(120))
        self.conv_net.add_module("Conv activation 9", nn.Mish())
        self.conv_net.add_module("Conv 10", nn.Conv2d(120, 110, 3, 1, 1))
        self.conv_net.add_module("SE 10", SE_Block(110))
        self.conv_net.add_module("Batchnorm 10", nn.BatchNorm2d(110))
        self.conv_net.add_module("Conv activation 10", nn.Mish())
        self.conv_net.add_module("Flattener", nn.Flatten())

        self.mlp = nn.Sequential()
        self.mlp.add_module("Linear 1", nn.Linear(7040, 1))
        self.mlp.add_module("Activation 1", nn.Sigmoid())

    def forward(self, x):
        a=self.conv_net(x)
        return self.mlp(self.conv_net(x))

    def count_parameters(self): return sum(p.numel() for p in self.parameters() if p.requires_grad)





model_name = "complex2_parrot"
model = ComplexModel(model_name)
state = torch.load(f"/content/drive/MyDrive/parrot/best_{model_name}.pickle", weights_only=True, map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()
print(model)
print("Current best model loaded successfully!")

Running on the GPU
Could not find tablebase
ComplexModel(
  (conv_net): Sequential(
    (Conv 1): Conv2d(1, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (SE 1): SE_Block(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (0): Linear(in_features=200, out_features=12, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=12, out_features=200, bias=False)
        (3): Sigmoid()
      )
    )
    (Batchnorm 1): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv activation): Mish()
    (Conv 2): Conv2d(200, 190, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (SE 2): SE_Block(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (0): Linear(in_features=190, out_features=11, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=11, out_features=190, bias=False)
        (3): Sigmoid()
      )
    )
    (Batchnorm

In [ ]:
# These are debug blocks, to walk through the search and look for possible bugs.

import helperfuncs
from helperfuncs import *
import numpy as np
def fast_board_to_boardmap(board):
    # Slower than piece_map() when there are less pieces on the board, but faster (~2x) in most cases.
    boards = [[0 for _ in range(8)] for _ in range(8)]
    for square in board.pieces(chess.PAWN, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["P"]
    for square in board.pieces(chess.PAWN, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["p"]
    for square in board.pieces(chess.KNIGHT, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["N"]
    for square in board.pieces(chess.KNIGHT, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["n"]
    for square in board.pieces(chess.BISHOP, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["B"]
    for square in board.pieces(chess.BISHOP, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["b"]
    for square in board.pieces(chess.ROOK, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["R"]
    for square in board.pieces(chess.ROOK, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["r"]
    for square in board.pieces(chess.QUEEN, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["Q"]
    for square in board.pieces(chess.QUEEN, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["q"]
    for square in board.pieces(chess.KING, chess.WHITE):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["K"]
    for square in board.pieces(chess.KING, chess.BLACK):
        idx = squareint_to_square(square)
        boards[idx[0]][idx[1]] = chr_to_num["k"]
    return [boards]


try:
    TABLEBASE = chess.syzygy.open_tablebase("/content/drive/MyDrive/parrot/tablebase_5pc")
    print("5 piece Syzygy endgame tablebase found.")
except:
    print("Could not find tablebase")
    TABLEBASE = None

TIMES_UP = -999
EXACT = 0
LOWERBOUND = 1
UPPERBOUND = 2

class Node:
    pass
class Node:

    def __init__(self, board: chess.Board, move: chess.Move | None, net: nn.Module, parent: Node | None, table: dict | None, depth=0):
        self.board = board
        self.move = move
        self.value = None
        self.parent = parent
        self.visits = 0
        self.depth = depth

        self.net = net
        self.children = []
        self.flag = None
        self.table = table

        try:
            self.capture = self.parent.board.is_capture(self.move)
        except:
            self.capture = False

        try:
            self.check = self.parent.board.is_check(self.move)
        except:
            self.check = False

        try:
            if self.move.promotion is not None:
                self.promotion = True
            else:
                self.promotion = False
        except:
            self.promotion = False


    def ucb(self, time_fraction, quiescent=0.02):
        bonus = 0
        if self.capture:
            bonus = quiescent
        elif self.check:
            bonus = quiescent / 2
        elif self.promotion:
            bonus = quiescent
        return self.value - (1 - time_fraction) * np.sqrt(np.log(self.parent.visits + 1) / (self.visits + 1)) - (bonus * (1 - time_fraction))

    def evaluate_nn(self):
        boardlist = fast_board_to_boardmap(self.board)
        if not self.board.turn:
            boardlist = np.rot90(boardlist, 2) * -1
            boardlist = boardlist.tolist()
        pos = torch.tensor(boardlist, device=device, dtype=torch.float).reshape(1, 1, 8, 8)
        with torch.no_grad():
            return self.net.forward(pos)

    def evaluate_position(self):
        if TABLEBASE and lt5(self.board):
            result = TABLEBASE.probe_wdl(self.board)
            if result == 2:
                return 1
            elif result == -2:
                return 0
            elif result in [-1, 0, 1]:
                return 0.5
        outcome = self.board.result(claim_draw=True)
        if outcome != "*":
            if (outcome == "1-0") and self.board.turn:
                return 1 + max((10 - self.depth) / 10, 0)
            elif (outcome == "0-1") and self.board.turn:
                return 0 - max((10 - self.depth) / 10, 0)
            elif (outcome == "1-0") and not self.board.turn:
                return 0 - max((10 - self.depth) / 10, 0)
            elif (outcome == "0-1") and not self.board.turn:
                return 1 + max((10 - self.depth) / 10, 0)
            elif outcome == "1/2-1/2":
                return 0.5
        return None

    def generate_children(self):
        all_positions = []

        evaled = []
        not_evaled = []
        helperfuncs.depth = max(helperfuncs.depth, self.depth + 1)
        blm = self.board.legal_moves
        helperfuncs.nodes += blm.count()
        for move in blm:
            newboard = self.board.copy()
            newboard.push(move)
            newnode = Node(newboard, move, self.net, self, self.table, depth=self.depth + 1)
            if newnode.board.halfmove_clock > 50:
                newnode.value = 0.5
                evaled.append(newnode)
            else:
                score = newnode.evaluate_position()
                if score is not None:
                    newnode.value = score
                    evaled.append(newnode)
                else:
                    boardlist = fast_board_to_boardmap(newboard)
                    if not newboard.turn:
                        boardlist = np.rot90(boardlist, 2) * -1
                        boardlist = boardlist.tolist()
                    all_positions.append(boardlist)
                    not_evaled.append(newnode)
            newnode.flag = EXACT

        pos = torch.tensor(all_positions, device=device, dtype=torch.float).reshape(len(not_evaled), 1, 8, 8)
        result = self.net.forward(pos)
        for i in range(len(not_evaled)):
            not_evaled[i].value = float(result[i])
            evaled.append(not_evaled[i])

        self.children = evaled


    def pns(self, start_time, time_for_this_move):
        while time.time() - start_time < time_for_this_move:

            # 1. Traverse tree with UCT + quiescence and decreasing exploration with time.
            target_node = self
            while target_node.children != []:
                target_node.visits += 1
                time_fraction = (time.time() - start_time) / time_for_this_move
                target_node = min(target_node.children, key=lambda child: child.ucb(time_fraction))

            # 2. Expansion and simulation
            target_node.generate_children()
            target_node.visits += 1

            # 3. Backpropagation
            while True:
                if target_node.children == []:
                    target_node.value = target_node.evaluate_position()
                    if target_node.value is None:
                        target_node.value = target_node.evaluate_nn()
                else:
                    target_node.value = 1 - min(target_node.children, key=lambda child: child.value).value
                if target_node.parent is not None:
                    target_node = target_node.parent
                else:
                    break

        # 4. Select move - UBFMS
        max_visits = max(self.children, key=lambda child: child.visits)
        print(max_visits.visits)
        selected_child = min(self.children, key=lambda child: child.value)
        print(selected_child.visits)
        print(self.value)
        return selected_child

Could not find tablebase


In [ ]:
# These are debug blocks, to walk through the search and look for possible bugs.

# r5k1/ppp1rppp/2np4/3B4/2PP2qP/8/PP1Q1PP1/R4RK1 w - - 0 1
# r5k1/ppp1rppp/2np4/3B4/2PP3q/8/PP3PP1/R2Q1RK1 w - - 0 1
# r2r2k1/5pp1/p1Q1p2p/P7/1p2P3/1B4N1/1PP2qPP/5R1K b - - 0 1
board = chess.Board("1rq2rk1/4ppbp/p2pbnp1/6N1/4PP2/1PN1Q2P/PB4P1/1R2R1K1 b - - 0 20")
node = Node(board, None, model, None, None, 0)
print(node.board, node.evaluate_nn())
print()
child = node.pns(time.time(), 4.483)
print(child.board, child.value, child.evaluate_nn())
print()

#board.push(chess.Move.from_uci("f2a7"))
#node = Node(board, None, model, None, None, 0)
#print(node.board, node.evaluate_nn())
#print()
#child = node.pns(time.time(), 10)
#print(child.board, child.value, child.evaluate_nn())
#print()
while node.children != []:
    child = min(node.children, key=lambda c: c.value)
    print(child.board, child.value, child.evaluate_nn())
    node = child
    print()


. r q . . r k .
. . . . p p b p
p . . p b n p .
. . . . . . N .
. . . . P P . .
. P N . Q . . P
P B . . . . P .
. R . . R . K . tensor([[0.5055]], device='cuda:0')

16
13
0.49437832832336426
. r q . . r k .
. . . . p p b p
p . . p b . p .
. . . . . . N .
. . . . P P n .
. P N . Q . . P
P B . . . . P .
. R . . R . K . 0.5056216716766357 tensor([[0.5354]], device='cuda:0')

. r q . . r k .
. . . . p p b p
p . . p b . p .
. . . . . . N .
. . . . P P n .
. P N . Q . . P
P B . . . . P .
. R . . R . K . 0.5056216716766357 tensor([[0.5354]], device='cuda:0')

. r q . . r k .
. . . . p p b p
p . . p b . p .
. . . N . . N .
. . . . P P n .
. P . . Q . . P
P B . . . . P .
. R . . R . K . 0.49437832832336426 tensor([[0.6950]], device='cuda:0')

. r q . . r k .
. . . . p p b p
p . . p . . p .
. . . b . . N .
. . . . P P n .
. P . . Q . . P
P B . . . . P .
. R . . R . K . 0.5056216716766357 tensor([[0.5056]], device='cuda:0')

